# 1.3 Data loading & Processing

**Refer to sections 1.1 and 1.2 for our analysis of the dataset, which provides the foundation for the techniques and methods, including imputation and feature engineering, discussed in section 1.3.**

**In this section, we will outline the process of creating a dataset that we believe is clean and high-quality for training the model.**

## Import library

In [1]:
import numpy as np 
import pandas as pd 
import os 
from concurrent.futures import ThreadPoolExecutor
from sklearn.impute import KNNImputer
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')

## Feature engineering helper function

### Time-series

In [2]:
def dictionary_of_statistics(data, time = None):
    try:
        #Handle empty dataframe
        if (data.empty or len(data.columns) == 0 or data is None):
            return {}
        
        if len(data.columns) == 0:
            return {}
        
        #Aggreate statistics for the dictionary
        stats_summary = data.agg(['mean', 'median', 'max', 'std']).to_dict()
        
        flattened_stats = {}
        for col, stats in stats_summary.items():
            for stat_name, value in stats.items():
                key = f"{stat_name}_{col}"
                if (time is not None):
                    key = f"{stat_name}_{col}_{time}"
                flattened_stats[key] = value
        
        return flattened_stats
    except Exception as e:
        return {}

#Feature engineering

def compute_time_features(data, day_start_hour=6, day_end_hour=18, expected_diff=5):
    
    """
    Compute and add time-related features to the DataFrame.

    Parameters:
    - data (pd.DataFrame): The input DataFrame containing 'time_of_day' in nanosecond and 'relative_date_PCIAT'.
    - day_start_hour (int): Hour to start the day period. Default is 8.
    - day_end_hour (int): Hour to end the day period. Default is 21.
    - expected_diff (int): Expected time difference between steps in seconds. Default is 5.
    
    """
    
    #From nanosecond to hour in a day
    data['time_of_day_hours'] = data['time_of_day'] / 1e9 / 3600
    data['day_time'] = data['relative_date_PCIAT'] + data['time_of_day_hours'] / 24
    
    #Categorize the day and night based on time data
    
    data['day_period'] = np.where(
        (data['time_of_day_hours'] >= day_start_hour) &
        (data['time_of_day_hours'] < day_end_hour),
        'day', 'night'
    )
    
    #Time difference beween steps
    #As the description, the time_of_day should represent the start of a 5s window over which the data was sampled
    #Calculate the time difference between each step
    data['time_diff'] = (data['day_time'].diff() * 86400).round(0) # seconds in a day
    data['measurement_after_gap'] = data['time_diff'] > expected_diff
    
def no_motion_periods(worn_data):
    """
    Find periods of no motion and give analytical insights in the data.

    Parameters:
    - data (pd.DataFrame): The input DataFrame containing 'time_of_day' and 'relative_date_PCIAT'.

    Returns:
    - pd.DataFrame: DataFrame with new features: 
    + total duration of no motion periods per day
    + the number of no motion periods per day.
    """
    
    #Calculate no motion periods
    no_motion = worn_data['enmo'] == 0
    motion_group = (
        (no_motion != no_motion.shift()) |
        (worn_data['measurement_after_gap'])
    ).cumsum()

    no_motion_periods = worn_data[no_motion].groupby(
        motion_group
    )['day_time'].agg(['min', 'max'])

    no_motion_periods['duration_sec'] = (
        (no_motion_periods['max'] - no_motion_periods['min']) * 86400
    ).round(0).astype(int)
    
    no_motion_periods['duration_sec'] += 5
    no_motion_periods['day'] = no_motion_periods['min'].astype(int)
    
    # Calculate daily statistics on no motion periods
    daily_stats = no_motion_periods.groupby(no_motion_periods['day']) \
        .agg(no_motion_duration=('duration_sec', 'sum'),
            no_motion_count=('duration_sec', 'size'))
    
    #Aggreate statistics for the dictionary
    return dictionary_of_statistics(daily_stats)

def circadian_rhythm_analysis(worn_data):
    
    """
    Make features capturing the variation in activity across the 24-hour cycle, 
    separately for day and night times (or wakefulness and sleep periods).
    
    Parameters:
    - data (pd.DataFrame): The input DataFrame containing 'time_of_day_hours', 'day_period' and 'relative_date_PCIAT'.
    
    Returns:
    - pd.DataFrame: 2 DataFrame, corresponding to day and night times,  with new features capturing the circadian rhythm of the wearer,
    + Standard deviation across hourly means per day
    + Peak hour of activity per day
    + Entropy of activity distribution per day
    """
    
    try:
        if (worn_data.empty or worn_data is None):
            return {}
    
        hourly_activity = worn_data.groupby(
            [worn_data['relative_date_PCIAT'].astype(int),
            worn_data['time_of_day_hours'].astype(int),
            worn_data['day_period']]
        )['enmo'].agg(['mean', 'max'])

        features = hourly_activity['mean'].groupby(
            ['relative_date_PCIAT', 'day_period']
        ).agg(
            std_across_hours='std',
            peak_hour=lambda x: x.idxmax()[1],
            entropy=lambda x: -(x / x.sum() * np.log(x / x.sum() + 1e-9)).sum()
        )
        
        # Safely extract day/night features
        try:
            day_features = features.xs('day', level='day_period')
        except:
            day_features = pd.DataFrame()
            
        try:    
            night_features = features.xs('night', level='day_period')
        except:
            night_features = pd.DataFrame()

        return dictionary_of_statistics(day_features, time="day") | dictionary_of_statistics(night_features, time='night')  
    except Exception as e:
        return {}

def physical_activity_analysis(worn_data):
        
    """
    Analyze the Moderate to Vigorous Physical Activity (MVPA) based on a threshold of ENMO values, 
    and calculate the duration of the detected MVPA activity bouts
    
    Parameters:
    - data (pd.DataFrame): The input DataFrame containing 'enmo', 'time_diff' and 'day_time'.
    
    Returns:
    - pd.DataFrame: DataFrame with new features capturing the physical activity level of the wearer,
    including:
    + Total duration of MVPA per day
    + Number of MVPA periods per day
    """
    # In order to classify physical activity as MVPA, we only retained activities that lasted at least 1 minute and met the criteria for the 100 mg (= 0.1g) threshold
    if worn_data is None or worn_data.empty:
        return {}
    
    mvpa_threshold = 0.1
    merge_gap = 60
    
    def merge_mvpa_groups(df, allowed_gap=60, merge_gap=60):
        last_mvpa_time = df['day_time'].where(df['is_mvpa']).ffill().shift()
        
        mvpa_time_diff = (
            (df['day_time'] - last_mvpa_time) * 86400
        ).round(0)
        
        mvpa_group = (
            (df['is_mvpa'] != df['is_mvpa'].shift()) |
            (df['time_diff'] >= allowed_gap)
        ).cumsum()
        
        is_mvpa_start = (
            (mvpa_group != mvpa_group.shift()) &
            df['is_mvpa']
        )
        
        group_increment = is_mvpa_start & (
            (mvpa_time_diff >= merge_gap) | last_mvpa_time.isnull()
        )
        
        merged_group = group_increment.cumsum()
        merged_group.loc[~df['is_mvpa']] = np.nan
        
        return merged_group
    
    worn_data['is_mvpa'] = worn_data['enmo'] > mvpa_threshold
    worn_data['mvpa_merged_group'] = merge_mvpa_groups(worn_data)

    mvpa_periods = worn_data[
        worn_data['is_mvpa']
    ].groupby('mvpa_merged_group')['day_time'].agg(['min', 'max'])

    mvpa_periods['duration_sec'] = (
        mvpa_periods['max'] - mvpa_periods['min']
    ) * 86400  # days to seconds

    mvpa_periods = mvpa_periods[mvpa_periods['duration_sec'] >= 60]
    mvpa_periods['duration_min'] = mvpa_periods['duration_sec'] / 60
    
    mvpa_periods['day'] = mvpa_periods['min'].astype(int)

    daily_stats = mvpa_periods.groupby(mvpa_periods['day']) \
        .agg(mvpa_total_duration=('duration_sec', 'sum'),
            mvpa_count_periods=('duration_sec', 'size'))
        
    return dictionary_of_statistics(daily_stats)
    
def activity_transition_analysis(worn_data):
        
    """
    The analysis to look at transitions between low, moderate and vigorous activity. To smooth out sudden, 
    short bursts of different activities, we filter out segments with a duration below a 1 minute threshold.
    
    Parameters:
    - data (pd.DataFrame): The input DataFrame containing 'enmo', 'time_diff' and 'day_time'.
    
    Returns:
    - pd.DataFrame: DataFrame with new features capturing the activity transitions of the wearer,
    + Total duration of different of activity per day
    + Number of different activity periods per day
    """
    mvpa_threshold = 0.1
    vig_threshold = 0.5
    worn_data['activity_type'] = pd.cut(
        worn_data['enmo'],
        bins=[-np.inf, mvpa_threshold, vig_threshold, np.inf],
        labels=['low', 'moderate', 'vigorous']
    )
    activity_group = (
        (worn_data['activity_type'] != worn_data['activity_type'].shift()) |
        (worn_data['measurement_after_gap'])
    ).cumsum()
    
    activity_periods = worn_data.groupby(activity_group).agg(
        min=('day_time', 'min'),
        max=('day_time', 'max'),
        activity_type=('activity_type', 'first')
    )
    activity_periods['duration_sec'] = (
        activity_periods['max'] - activity_periods['min']
    ) * 86400 + 5 

    activity_periods = activity_periods[activity_periods['duration_sec'] >= 60]
    activity_periods['duration_min'] = activity_periods['duration_sec'] / 60
    
    activity_periods['day'] = activity_periods['min'].astype(int)
    activity_periods['transition_num'] = (
        activity_periods.groupby('day')['activity_type']
        .apply(lambda x: (x != x.shift()).cumsum())
        .reset_index(level=0, drop=True)
    )
    
    low_activity = activity_periods[activity_periods['activity_type'] == 'low'].groupby('day').agg(
        low_act_total_duration=('duration_sec', 'sum'),
        low_act_count_periods=('duration_sec', 'size')
    )
    
    moderate_activity = activity_periods[activity_periods['activity_type'] == 'moderate'].groupby('day').agg(
            moderate_act_total_duration=('duration_sec', 'sum'),
            moderate_act_count_periods=('duration_sec', 'size')
    )

        
    daily_transitions = activity_periods.groupby('day').agg(
        count_transitions=('transition_num', 'max')
    )
    
    return dictionary_of_statistics(low_activity) | dictionary_of_statistics(moderate_activity) | dictionary_of_statistics(daily_transitions)

def activity_light_exposure(worn_data):
        
    """
    Analyze the correlation between light exposure and physical activity level.
    
    Parameters:
    - data (pd.DataFrame): The input DataFrame containing 'light' and 'enmo'.
    
    Returns:
    - float: The correlation between light exposure and physical activity level.
    """
    correlation_light_enmo = worn_data[['light', 'enmo']].corr().iloc[0, 1]
    return {'correlation_light_enmo': correlation_light_enmo}

def process_file(file_path, participant_id):
    try:
        data = pd.read_parquet(file_path)
        
        if data.empty:
            return {'id': participant_id}
        
        required_columns = ['time_of_day', 'relative_date_PCIAT', 'enmo', 'non-wear_flag', 'light']
        if not all(col in data.columns for col in required_columns):
            return {'id': participant_id}

        # Compute time features
        compute_time_features(data)
            
        # Calculate the percentage of non-worn time
        non_wear_percentage = (data['non-wear_flag'].sum() / len(data)) * 100
        
        # Filter out the worn data
        worn_data = data[data['non-wear_flag'] == 0]
        
        if (worn_data.empty):
            return {'id': participant_id}
        
        # recalculate time difference between rows and measurement_after_gap flag in the worn data
        expected_diff = 5
        worn_data['time_diff'] = (worn_data['day_time'].diff() * 86400).round(0)
        worn_data['measurement_after_gap'] = worn_data['time_diff'] > expected_diff

        # Compute no motion periods
        no_motion_stats = no_motion_periods(worn_data)
        # Circadian rhythm analysis
        circadian_rhythm_stats = circadian_rhythm_analysis(worn_data)
        # Physical activity analysis
        physical_activity_stats = physical_activity_analysis(worn_data)
        # Activity transition analysis
        activity_transition_stats = activity_transition_analysis(worn_data)
        # Compute activity light correlation
        activity_light_stats = activity_light_exposure(worn_data)

        return {
            'id': participant_id,
        } | no_motion_stats | circadian_rhythm_stats | physical_activity_stats | activity_transition_stats | activity_light_stats
    
    except Exception as e:
        return {'id': participant_id}

def load_time_series(dir_name):
    
    participant_ids = os.listdir(dir_name)
    
    with ThreadPoolExecutor() as executor:
        #tqdm: Wraps the executor.map iterable with tqdm -> Show a progress bar indicating the processing status
        results = list(tqdm(executor.map(lambda x: process_file(os.path.join(dir_name, x, 'part-0.parquet'), x), participant_ids), total=len(participant_ids)))

    df = pd.DataFrame(results)
    # Replace inf and -inf with NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Replace NaN with 0
    df.fillna(0, inplace=True)
    
    return df

### Tabular

In [3]:
def feature_engineering(df):
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
    df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat']
    df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
    df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
    df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
    df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
    df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
    df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
    df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
    df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
    df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
    df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']
    df['BMI_PHR'] = df['Physical-BMI'] * df['Physical-HeartRate']

    # Replace any remaining inf values with NaN
    df = df.replace([np.inf, -np.inf], np.nan)
    
    return df

## Data loading

### Load time-series data

In [4]:
train_ts = load_time_series("../data/series_train.parquet")
test_ts = load_time_series("../data/series_test.parquet")

100%|██████████| 2/2 [00:00<00:00,  7.27it/s]


**A small modification enables consistant formats for both time-series and tabular data.**

In [5]:
train_ts['id'] = train_ts['id'].str.replace('id=', '')
test_ts['id'] = test_ts['id'].str.replace('id=', '')

### Load tabular data

In [7]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

**As mentioned in section 1.1, the tabular data contains columns that appear only in the training set, with the exception of "sii," which is the target variable. Specifically, the columns related to the PCIAT calculation are present only in the training set and must be excluded to ensure data consistency between the training and testing sets.**

In [8]:
train_cols = set(train.columns)
test_cols = set(test.columns)
dif_cols = sorted(list(train_cols - test_cols))

dif_cols

['PCIAT-PCIAT_01',
 'PCIAT-PCIAT_02',
 'PCIAT-PCIAT_03',
 'PCIAT-PCIAT_04',
 'PCIAT-PCIAT_05',
 'PCIAT-PCIAT_06',
 'PCIAT-PCIAT_07',
 'PCIAT-PCIAT_08',
 'PCIAT-PCIAT_09',
 'PCIAT-PCIAT_10',
 'PCIAT-PCIAT_11',
 'PCIAT-PCIAT_12',
 'PCIAT-PCIAT_13',
 'PCIAT-PCIAT_14',
 'PCIAT-PCIAT_15',
 'PCIAT-PCIAT_16',
 'PCIAT-PCIAT_17',
 'PCIAT-PCIAT_18',
 'PCIAT-PCIAT_19',
 'PCIAT-PCIAT_20',
 'PCIAT-PCIAT_Total',
 'PCIAT-Season',
 'sii']

## Data preprocessing

### Recalculate 'sii' values

**Few SII scores are still derived from the sum of NAN values in PICAT questions, leading to potentially invalid SII values. The below cell tries to estimate the severity of internet usage SII based on current SII and the maximum possible SII.**

In [9]:
PCIAT_cols = [f'PCIAT-PCIAT_{i+1:02d}' for i in range(20)]

#Recalculates the SII value based on the the current PCIAT values and the possible maximum PCIAT values
def recalculate_sii(row):
    value = 0
    if (not pd.isna(row['PCIAT-PCIAT_Total'])):
        value = row['PCIAT-PCIAT_Total']
        
    max_possible = value + row[PCIAT_cols].isna().sum() * 5
    
    if value <= 30 and max_possible <= 30:
        return 0
    elif 31 <= value <= 49 and max_possible <= 49:
        return 1
    elif 50 <= value <= 79 and max_possible <= 79:
        return 2
    elif value >= 80 and max_possible >= 80:
        return 3
    
    return np.nan

train['recalc_sii'] = train.apply(recalculate_sii, axis=1)
train['sii'] = train['recalc_sii']
train.drop(columns='recalc_sii', inplace=True)

### Data merging

In [10]:
train = pd.merge(train, train_ts, how="left", on='id')
test = pd.merge(test, test_ts, how="left", on='id')

**Take a look at the test set after grafting encoded time-series data. As shown below, the participants who didn't wear device have features related to time-series values are NaN. To handle, we will fill all NaN values via KNNImputer trained on data in training set from those who worn devices.**

In [11]:
test.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,std_moderate_act_total_duration,mean_moderate_act_count_periods,median_moderate_act_count_periods,max_moderate_act_count_periods,std_moderate_act_count_periods,mean_count_transitions,median_count_transitions,max_count_transitions,std_count_transitions,correlation_light_enmo
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,61.352805,1.166667,1.0,2.0,0.408248,1.5,1.0,3.0,0.888523,0.129729
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Impute missing values for numeric columns

**Initialize a KNNImputer for handling missing values. Statistically, KNNImputer outweights SimpleImputer in this context. However, it encounters the computationally expensive cost, and the requirement of hyperparameter K (K value should be considered to change later). The imputation should be applied on both train and test set. However, to avoid data leakage, the imputer has to be trained on data from training set, then appied to both train and test set**

In [12]:
def numeric_imputation(train_df, test_df):
    numeric_cols = train_df.select_dtypes(include=['float64', 'int64', 'int32', 'float32']).columns.tolist()
    
    numeric_cols = [col for col in numeric_cols if col not in dif_cols]

    imputer = KNNImputer(n_neighbors=5)

    #Fit on training set
    imputer.fit(train_df[numeric_cols])

    #Transform both
    train_df[numeric_cols] = imputer.transform(train_df[numeric_cols])
    test_df[numeric_cols] = imputer.transform(test_df[numeric_cols])

    return train_df, test_df

In [13]:
train, test = numeric_imputation(train, test)

In [14]:
train.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,std_moderate_act_total_duration,mean_moderate_act_count_periods,median_moderate_act_count_periods,max_moderate_act_count_periods,std_moderate_act_count_periods,mean_count_transitions,median_count_transitions,max_count_transitions,std_count_transitions,correlation_light_enmo
0,00008ff9,Fall,5.0,0.0,Winter,51.0,Fall,16.877316,46.00,50.8,...,386.547341,5.975914,4.8,17.0,4.263757,8.655200,7.4,22.0,5.767433,0.132322
1,000fd460,Summer,9.0,0.0,NaN,68.8,Fall,14.035590,48.00,46.0,...,309.145707,6.837425,5.6,13.6,2.756057,6.126791,5.0,17.6,4.862391,0.178144
2,00105258,Summer,10.0,1.0,Fall,71.0,Fall,16.648696,56.50,75.6,...,255.959519,4.193237,3.7,9.0,2.397566,4.494649,3.4,14.6,3.774495,0.210945
3,00115b9f,Winter,9.0,0.0,Fall,71.0,Summer,18.292347,56.00,81.6,...,61.352805,1.166667,1.0,2.0,0.408248,1.500000,1.0,3.0,0.888523,0.129729
4,0016bb22,Spring,18.0,1.0,Summer,71.0,NaN,26.713639,62.54,123.8,...,486.388753,5.023333,4.2,11.6,2.954711,6.803429,6.4,16.2,4.194390,0.149184


### Impute missing values for categorical columns

In [16]:
#Get categorical columns
cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
          'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
          'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

#File the nan values in categorical collumns with 'Missing'
def update(df):
    global cat_c
    for c in cat_c: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df

train = update(train)
test = update(test)

#Convert from categorical to numeric 
def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

for col in cat_c:
    mapping = create_mapping(col, train)
    mappingTe = create_mapping(col, test)
    
    train[col] = train[col].replace(mapping).astype(int)
    test[col] = test[col].replace(mappingTe).astype(int)

In [17]:
train.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,std_moderate_act_total_duration,mean_moderate_act_count_periods,median_moderate_act_count_periods,max_moderate_act_count_periods,std_moderate_act_count_periods,mean_count_transitions,median_count_transitions,max_count_transitions,std_count_transitions,correlation_light_enmo
0,00008ff9,0,5.0,0.0,0,51.0,0,16.877316,46.00,50.8,...,386.547341,5.975914,4.8,17.0,4.263757,8.655200,7.4,22.0,5.767433,0.132322
1,000fd460,1,9.0,0.0,1,68.8,0,14.035590,48.00,46.0,...,309.145707,6.837425,5.6,13.6,2.756057,6.126791,5.0,17.6,4.862391,0.178144
2,00105258,1,10.0,1.0,2,71.0,0,16.648696,56.50,75.6,...,255.959519,4.193237,3.7,9.0,2.397566,4.494649,3.4,14.6,3.774495,0.210945
3,00115b9f,2,9.0,0.0,2,71.0,1,18.292347,56.00,81.6,...,61.352805,1.166667,1.0,2.0,0.408248,1.500000,1.0,3.0,0.888523,0.129729
4,0016bb22,3,18.0,1.0,3,71.0,2,26.713639,62.54,123.8,...,486.388753,5.023333,4.2,11.6,2.954711,6.803429,6.4,16.2,4.194390,0.149184


## Feature engineering

In [18]:
train = feature_engineering(train)
#Remove rows which has got fewer than 10 non-null values
train = train.dropna(thresh=10, axis=0)
test = feature_engineering(test)

train = train.drop('id', axis=1)
test  = test .drop('id', axis=1)

**As mentioned in section 1.1, many features in tabular data have a significant number of missing values. For example, columns like PAQ_A-Season and PAQ_A-PAQ_A_Total are missing nearly 90% of their values. Therefore, it is essential to select columns that are relevant and consistent, as this will enhance the efficiency of model building. The cell below highlights the features we have identified and evaluated as important.**

In [21]:
featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
                'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
                'PreInt_EduHx-computerinternet_hoursday', 'sii',
                'BMI_Age','Internet_Hours_Age','BMI_Internet_Hours',
                'BFP_BMI', 'FFMI_BFP', 'FMI_BFP', 'LST_TBW', 'BFP_BMR', 'BFP_DEE', 'BMR_Weight', 'DEE_Weight',
                'SMM_Height', 'Muscle_to_Fat', 'Hydration_Status', 'ICW_TBW','BMI_PHR']

featuresCols += train_ts.columns.tolist()
featuresCols.remove('id')

train = train[featuresCols]

featuresCols.remove('sii')
test = test[featuresCols]

In [22]:
#Check and replace any INF values with NaN
if np.any(np.isinf(train)):
    train = train.replace([np.inf, -np.inf], np.nan)

**The cell below checks for any NaN (Not a Number) values in the training dataset. Even though imputation has been performed previously, it is still possible for unwanted values like INF (infinity) or NaN to be generated during the feature engineering process. Rechecking the data ensures that the dataset is completely clean, which helps to avoid issues during model building.**

In [23]:
df = train
# Get columns with NaN values
nan_columns = df.columns[df.isna().any()].tolist()

# Calculate number of NaN values per column
nan_counts = df[nan_columns].isna().sum()

# Sort by number of NaN values (descending)
nan_counts_sorted = nan_counts.sort_values(ascending=False)

# Display results
print("\nColumns with NaN values:")
print("-" * 50)
for col, count in nan_counts_sorted.items():
    total = len(df)
    percentage = (count/total * 100)
    print(f"{col:<30} {count:>7} NaN values ({percentage:>6.2f}%)")

print(f"\nTotal columns with NaN values: {len(nan_columns)}")


Columns with NaN values:
--------------------------------------------------
sii                               1241 NaN values ( 31.34%)
BMR_Weight                          63 NaN values (  1.59%)
DEE_Weight                          63 NaN values (  1.59%)
Hydration_Status                    63 NaN values (  1.59%)

Total columns with NaN values: 4


**Aside from the sii column, which has nearly one-third of its data missing (we will discuss how to handle this in the next section 2. and 3.), other columns like BMR_Weight, DEE_Weight, and Hydration_Status also have missing data. These columns are new compared to the original dataset and were created during the feature engineering process. However, with the percentage of missing data being only 1.59%, we decided to remove the rows containing NaN values. This small fraction of data is unlikely to affect the quality and reliability of entire dataset.**

In [24]:
train = train.dropna(subset=['BMR_Weight', 'DEE_Weight', 'Hydration_Status'])

In [25]:
print(train.shape, test.shape)

(3897, 136) (20, 135)


In [26]:
train.head()

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,...,std_moderate_act_total_duration,mean_moderate_act_count_periods,median_moderate_act_count_periods,max_moderate_act_count_periods,std_moderate_act_count_periods,mean_count_transitions,median_count_transitions,max_count_transitions,std_count_transitions,correlation_light_enmo
0,0,5.0,0.0,0,51.0,0,16.877316,46.00,50.8,23.2,...,386.547341,5.975914,4.8,17.0,4.263757,8.655200,7.4,22.0,5.767433,0.132322
1,1,9.0,0.0,1,68.8,0,14.035590,48.00,46.0,22.0,...,309.145707,6.837425,5.6,13.6,2.756057,6.126791,5.0,17.6,4.862391,0.178144
2,1,10.0,1.0,2,71.0,0,16.648696,56.50,75.6,25.0,...,255.959519,4.193237,3.7,9.0,2.397566,4.494649,3.4,14.6,3.774495,0.210945
3,2,9.0,0.0,2,71.0,1,18.292347,56.00,81.6,26.0,...,61.352805,1.166667,1.0,2.0,0.408248,1.500000,1.0,3.0,0.888523,0.129729
4,3,18.0,1.0,3,71.0,2,26.713639,62.54,123.8,33.6,...,486.388753,5.023333,4.2,11.6,2.954711,6.803429,6.4,16.2,4.194390,0.149184
